In [3]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

Using backend: pytorch


In [4]:
dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [3]:
g.edata

{}

In [8]:
from dgl.nn import GraphConv

In [9]:
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)
        
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h
    
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

In [10]:
labels = g.ndata['label']

In [11]:
labels

tensor([3, 4, 4,  ..., 3, 3, 3])

In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

best_val_acc = 0
best_test_acc = 0

features = g.ndata['feat']
labels = g.ndata['label']

train_mask = g.ndata['train_mask']
val_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']

for e in range(1):
    logits = model(g, features)

    pred = logits.argmax(1)

    loss = F.cross_entropy(logits[train_mask], labels[train_mask])

    train_acc = (pred[train_mask]==labels[train_mask]).float().mean()
    val_acc = (pred[val_mask]==labels[val_mask]).float().mean()
    test_acc = (pred[test_mask]==labels[test_mask]).float().mean()

    if best_val_acc < val_acc:
        best_val_acc = val_acc

    if best_test_acc < test_acc:
        best_test_acc = test_acc


    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
            e, loss, val_acc, best_val_acc, test_acc, best_test_acc))


In epoch 0, loss: 1.945, val acc: 0.094 (best 0.094), test acc: 0.127 (best 0.127)


In [18]:
labels[train_mask]

tensor([3, 4, 4, 0, 3, 2, 0, 3, 3, 2, 0, 0, 4, 3, 3, 3, 2, 3, 1, 3, 5, 3, 4, 6,
        3, 3, 6, 3, 2, 4, 3, 6, 0, 4, 2, 0, 1, 5, 4, 4, 3, 6, 6, 4, 3, 3, 2, 5,
        3, 4, 5, 3, 0, 2, 1, 4, 6, 3, 2, 2, 0, 0, 0, 4, 2, 0, 4, 5, 2, 6, 5, 2,
        2, 2, 0, 4, 5, 6, 4, 0, 0, 0, 4, 2, 4, 1, 4, 6, 0, 4, 2, 4, 6, 6, 0, 0,
        6, 5, 0, 6, 0, 2, 1, 1, 1, 2, 6, 5, 6, 1, 2, 2, 1, 5, 5, 5, 6, 5, 6, 5,
        5, 1, 6, 6, 1, 5, 1, 6, 5, 5, 5, 1, 5, 1, 1, 1, 1, 1, 1, 1])

In [19]:
logits[train_mask][0]

tensor([-0.6872, -0.3376, -0.9671,  5.0462, -0.0324,  0.0244, -1.9166],
       grad_fn=<SelectBackward>)

In [21]:
labels[train_mask]

tensor([3, 4, 4, 0, 3, 2, 0, 3, 3, 2, 0, 0, 4, 3, 3, 3, 2, 3, 1, 3, 5, 3, 4, 6,
        3, 3, 6, 3, 2, 4, 3, 6, 0, 4, 2, 0, 1, 5, 4, 4, 3, 6, 6, 4, 3, 3, 2, 5,
        3, 4, 5, 3, 0, 2, 1, 4, 6, 3, 2, 2, 0, 0, 0, 4, 2, 0, 4, 5, 2, 6, 5, 2,
        2, 2, 0, 4, 5, 6, 4, 0, 0, 0, 4, 2, 4, 1, 4, 6, 0, 4, 2, 4, 6, 6, 0, 0,
        6, 5, 0, 6, 0, 2, 1, 1, 1, 2, 6, 5, 6, 1, 2, 2, 1, 5, 5, 5, 6, 5, 6, 5,
        5, 1, 6, 6, 1, 5, 1, 6, 5, 5, 5, 1, 5, 1, 1, 1, 1, 1, 1, 1])

In [16]:
loss

tensor(0.0911, grad_fn=<NllLossBackward>)

In [25]:
len(train_mask)

2708

In [13]:
train(g, model)


In epoch 0, loss: 1.947, val acc: 0.094 (best 0.094), test acc: 0.103 (best 0.103)
In epoch 5, loss: 1.892, val acc: 0.604 (best 0.604), test acc: 0.640 (best 0.640)
In epoch 10, loss: 1.808, val acc: 0.600 (best 0.612), test acc: 0.634 (best 0.641)
In epoch 15, loss: 1.698, val acc: 0.620 (best 0.620), test acc: 0.649 (best 0.649)
In epoch 20, loss: 1.564, val acc: 0.672 (best 0.672), test acc: 0.698 (best 0.698)
In epoch 25, loss: 1.411, val acc: 0.724 (best 0.724), test acc: 0.741 (best 0.741)
In epoch 30, loss: 1.242, val acc: 0.724 (best 0.726), test acc: 0.738 (best 0.741)
In epoch 35, loss: 1.069, val acc: 0.734 (best 0.734), test acc: 0.754 (best 0.754)
In epoch 40, loss: 0.899, val acc: 0.748 (best 0.748), test acc: 0.760 (best 0.760)
In epoch 45, loss: 0.743, val acc: 0.752 (best 0.752), test acc: 0.770 (best 0.770)
In epoch 50, loss: 0.607, val acc: 0.760 (best 0.760), test acc: 0.776 (best 0.776)
In epoch 55, loss: 0.492, val acc: 0.764 (best 0.764), test acc: 0.780 (best 0

In [14]:
F.cross_entropy(logits[train_mask], labels[train_mask])

NameError: name 'logits' is not defined

In [26]:
model

GCN(
  (conv1): GraphConv(in=1433, out=16, normalization=both, activation=None)
  (conv2): GraphConv(in=16, out=7, normalization=both, activation=None)
)

In [30]:
logits.shape

torch.Size([2708, 7])

In [31]:
sum(train_mask), sum(val_mask)

(tensor(140), tensor(500))

In [32]:
sum(test_mask)

tensor(1000)

In [33]:
len(test_mask)

2708

In [36]:
g.edges(form='all')

(tensor([   0,    0,    0,  ..., 2707, 2707, 2707]),
 tensor([ 633, 1862, 2582,  ...,  598, 1473, 2706]),
 tensor([    0,     1,     2,  ..., 10553, 10554, 10555]))

In [37]:
g.nodes()

tensor([   0,    1,    2,  ..., 2705, 2706, 2707])

In [4]:
train_mask = g.ndata['train_mask']

In [5]:
train_mask

tensor([ True,  True,  True,  ..., False, False, False])